In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


['sample_submission.csv', 'train.csv', 'test.csv']


In [2]:
%cd ../input/

/kaggle/input


In [3]:

df = pd.read_csv('train.csv')

y = df['label']
X = df.drop(['label'],axis = 1)
test = pd.read_csv('test.csv')
n3 = test.values



In [4]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size= 0.15)
y_train = y_train.values
y_val = y_val.values
X_train = X_train.values
X_val = X_val.values

#change cell to code to run SKlearn neural net classifier
from sklearn.neural_network import MLPClassifier
solvers = ['lbfgs', 'sgd', 'adam']
for solver in solvers:
    clf = MLPClassifier(solver=solver,activation='logistic', alpha=1e-5,hidden_layer_sizes=(100), random_state=1)
    clf.fit(X_train,y_train)
    print(clf.score(X_val,y_val))

In [5]:
def Reshape2d(x):
    x = np.reshape(x,(-1,28,28))
    return x
def next_batch(x,y,batch_size):
    
    while True:
        try:
            i +=1
        except:
            i=0
        if i>=len(x)//batch_size:
            i=0
        yield (np.reshape(x[i*batch_size:(i+1)*batch_size],(batch_size,1,28,28)), y[i*batch_size:(i+1)*batch_size])

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D,MaxPooling2D,Flatten
from keras.optimizers import SGD
model = Sequential()

model.add(Conv2D(32,kernel_size= 5 ,strides = 1,data_format="channels_first",activation = 'relu',input_shape = (1,28,28)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_first'))
model.add(Conv2D(64,(3,3),strides = 1,data_format="channels_first",activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_first'))
model.add(Flatten(data_format="channels_first"))
model.add(Dropout(rate=0.5))
model.add(Dense(400,activation='sigmoid'))
#model.add(Dropout(rate=0.4))
model.add(Dense(10,activation='sigmoid'))
model.compile(optimizer='sgd',loss = 'categorical_crossentropy',metrics=['accuracy'])

Using TensorFlow backend.


In [7]:
#keras.backend.shape()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 24, 24)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 12, 12)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 10, 10)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 5, 5)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 400)               640400    
__________

In [8]:
X_train = Reshape2d(X_train)
X_val = Reshape2d(X_val)
#print(type(X_train))
from keras.utils.np_utils import to_categorical  

y_train = to_categorical(y_train,num_classes=10)
y_val = to_categorical(y_val,num_classes=10)

In [9]:
print(X_val.shape)
print(y_val.shape)
y_val

(6300, 28, 28)
(6300, 10)


array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [10]:

batch_size = 64
model.fit_generator(next_batch(X_train,y_train,batch_size),epochs = 100,validation_data=(np.reshape(X_val,(-1,1,28,28)),y_val),steps_per_epoch=len(X_train)//batch_size)

Epoch 1/100
557/557 [==============================] - 7s 12ms/step - loss: 1.1197 - acc: 0.6839 - val_loss: 0.2580 - val_acc: 0.9360
Epoch 2/100
557/557 [==============================] - 3s 5ms/step - loss: 0.3108 - acc: 0.9284 - val_loss: 0.1430 - val_acc: 0.9610
Epoch 3/100
557/557 [==============================] - 3s 5ms/step - loss: 0.2035 - acc: 0.9486 - val_loss: 0.1078 - val_acc: 0.9675
Epoch 4/100
557/557 [==============================] - 3s 5ms/step - loss: 0.1610 - acc: 0.9580 - val_loss: 0.0922 - val_acc: 0.9690
Epoch 5/100
557/557 [==============================] - 3s 5ms/step - loss: 0.1344 - acc: 0.9653 - val_loss: 0.0829 - val_acc: 0.9740
Epoch 6/100
557/557 [==============================] - 3s 6ms/step - loss: 0.1195 - acc: 0.9673 - val_loss: 0.0702 - val_acc: 0.9771
Epoch 7/100
557/557 [==============================] - 3s 6ms/step - loss: 0.1062 - acc: 0.9711 - val_loss: 0.0704 - val_acc: 0.9773
Epoch 8/100
557/557 [==============================] - 3s 5ms/step -

In [11]:
try:
    counter +=1
except:
    counter = 0
model.save('../working/model0'+str(counter))

In [12]:
!ls ../working/model1

ls: cannot access '../working/model1': No such file or directory


In [13]:
n3 = np.reshape(n3,(-1,1,28,28))

z= model.predict(n3)
z = [np.argmax(r) for r in z]
z1=pd.DataFrame({'ImageId': np.arange(1,len(z)+1) ,'label': z})
z1.to_csv("../working/foo.csv",index=False)